# McStas Union components

The McStas Union components is a collection of McStas components that work together to simulate complex systems with scattering in volumes.

Developed by Mads Bertelsen during PhD at University of Copenhagen and development has continued at ESS DMSC.

![alt](figures/power_point_figures/Slide1.png)

## Union components: Distributed responsibility

![alt](figures/power_point_figures/Slide2.png)

## Physics
Introduction to process components and *Union_make_material*.

In [1]:
import mcstasscript as ms
%matplotlib widget
instrument = ms.McStas_instr("Union_demo")

incoherent = instrument.add_component("incoherent", "Incoherent_process")
incoherent.sigma = 2.5
incoherent.unit_cell_volume = 13.8

inc_material = instrument.add_component("inc_material", "Union_make_material")
inc_material.my_absorption = 1.2
inc_material.process_string = '"incoherent"'

instrument.show_diagram()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Absorption cross section given as inverse penetration depth $\mu$ from $I(z) = e^{-\mu z}$. Unit of 1/m.

### Aluminium description
Fully modular system, can include as many processes in a material as desired.

In [2]:
instrument = ms.McStas_instr("Union_demo")

Al_incoherent = instrument.add_component("Al_incoherent", "Incoherent_process")
Al_incoherent.set_parameters(sigma=4*0.0082, unit_cell_volume=66.4)

Al_powder = instrument.add_component("Al_powder", "Powder_process")
Al_powder.reflections = '"Al.laz"'

Al = instrument.add_component("Al", "Union_make_material")
Al.process_string = '"Al_incoherent,Al_powder"'
Al.my_absorption = 100*4*0.231/66.4

instrument.show_diagram()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Single crystal description
No limits on the number of user defined materials either, now we have aluminium powder and diamond crystal.

In [3]:
process = instrument.add_component("diamond_incoherent", "Incoherent_process")
process.set_parameters(sigma=8*0.001, unit_cell_volume=45.39)

crystal_process = instrument.add_component("diamond_crystal", "Single_crystal_process")
crystal_process.set_parameters(mosaic=20, reflections='"C_diamond.lau"',
                               ax=3.567, by=3.567, cz=3.567,)

YBaCuO = instrument.add_component("diamond", "Union_make_material")
YBaCuO.process_string = '"diamond_incoherent,diamond_crystal"'
YBaCuO.my_absorption = 100*8*0.0035/45.39

instrument.show_diagram()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Physics
A process just needs to describe the probability for scattering and handle the scattering event. Here are the currently available processes.

| Process                 | Description                                 | Contributor                 |
| :---                    |    :----                                    |    :----                    |
| Incoherent_process      | Isotropic incoherent                        | From McStas Incoherent      |
| Powder_process          | Powder Bragg                                | From McStas PowderN         |
| Single_crystal_process  | Single crystal Bragg                        | From McStas Single_crystal  |
| Phonon_simple_process   | Phonon on fcc lattice                       | From McStas Phonon_simple   |
| NCrystal_process        | NCrystal import                             | From McStas NCrystal_sample |
| AF_HB_1D_process        | Antiferromagnetic 1D heisenberg spin chain  | Mads Bertelsen              |
| Texture_process         | Textured powder                             | Victor Laliena              |
| IncoherentPhonon_process| Phonon powder                               | Victor Laliena              |

It is even possible to add several instances of the same process to a material, for example to create twinned crystals, each with a phonon process oriented to that lattice.

## Geometry
Geometry components are placed in space and describe some geometry, they use a material definition to inherit their physical properties.

| Geometry                | Description                                 | Contributor     |
| :---                    |    :----                                    |    :----        |
| Union_sphere            | Sphere                                      | Mads Bertelsen  |
| Union_cylinder          | Finite cylinder                             | Mads Bertelsen  |
| Union_box               | Box similar to a guide element              | Mads Bertelsen  |
| Union_cone              | Capped cone                                 | Martin Olsen    |
| Union_mesh              | Concave stl file (only works alone)         | Martin Olsen    |


### Geometry example
Lets make a diamond sphere

In [4]:
sphere = instrument.add_component("sphere", "Union_sphere")
sphere.radius = 0.01
sphere.material_string = '"diamond"'
sphere.set_AT([0, 0, 1])

instrument.show_diagram()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
print(sphere)

COMPONENT sphere = Union_sphere
  material_string = "diamond" []
  priority : Required parameter not yet specified
  radius = 0.01 [m]
AT [0, 0, 1] ABSOLUTE


### Geometry priority
Priority is a required parameter that handles how geometries overlap. Any point in space corresponds to one material, so when a point is in two or more geometries, the material with the highest priority is used.

In [6]:
%matplotlib widget
import geometry_example
geometry_example.show()

### Adding am aluminium shell around our diamond sphere

In [7]:
sphere.priority = 10

shell = instrument.add_component("shell", "Union_cylinder")
shell.set_parameters(radius=0.05, yheight=0.12,
                     material_string = '"Al"', priority=3,
                     p_interact=0.2)
shell.set_AT(0, RELATIVE=sphere)

shell_vacuum = instrument.add_component("shell_vacuum", "Union_cylinder")
shell_vacuum.set_parameters(radius=shell.radius - 0.01, yheight=shell.yheight - 0.01,
                            material_string = '"Vacuum"', priority=5)
shell_vacuum.set_AT([0, 0, 0], RELATIVE=shell)

![alt](figures/power_point_figures/Slide5.png)

## Master component
There are four different types of Union components that must be present in an instrument to perform a simulation:

- process components
- make_material component
- geometry components
- master

The *Union_master* component contains the engine that actually performs the simulation. It is the location of the Union master that decides where in the sequence of McStas components the Union system is simulated.

![alt](figures/power_point_figures/Slide4.png)

### Placing the source
Since the master component will perform the simulation in the McStas component sequence, lets place a source before the master.

In [8]:
source = instrument.add_component("source", "Source_div")
source.set_parameters(focus_aw=0.3, focus_ah=0.3,
                      xwidth=0.005, yheight=0.005,
                      flux=1E12)
source.set_AT(-1, RELATIVE=sphere)

source.lambda0 = instrument.add_parameter("wavelength", value=5.0,
                                          comment="Wavelength in [Ang]")
source.dlambda = instrument.add_parameter("wavelength_half_width", value=4.5,
                                          comment="Wavelength half width in [Ang]")

## Place a master
The master performs a full multiple scattering simulation of all geometries defined before it.

It is possible to have several masters in an instrument, the second would simulate the geometries defined between the two masters.

In McStas 3.X it is also necessary to include a Union_init and Union_stop component before/after any Union components. 

In [9]:
master = instrument.add_component("master", "Union_master")

In [10]:
instrument.show_diagram()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Visualizing the instrument
The Master component is required to draw the Union system.

In [11]:
instrument.show_instrument(width=1000, height=800)

### Example of system simulated with Union components

![alt](figures/cryostat_geometry.png)

## Logger components
Logger components record scattering in the simulated Union system.

| Logger component           | Dimensions                            |
| :---                       |    :----                              |
| Union_logger_1D            | time, magnitude of scattering vector  |
| Union_logger_2DQ           | 2D scattering vector                  |
| Union_logger_2D_kf         | 2D final wavevector                   |
| Union_logger_2D_kf_time    | 2D final wavevector and time          |
| Union_logger_2D_space      | 2D space                              |
| Union_logger_2D_space_time | 2D space and time                     |
| Union_logger_3D_space      | 3D space                              |

Some inputs are common for all Union logger components.

| Parameter                  | Explanation                                                  |
| :---                       |    :----                                                     |
| target_geometry            | Only record for given list of geometries                     |
| target_process             | Further select processes (if geometry selected)              |
| order_total                | Record only the n'th scattering of rays                      |
| order_volume               | Record only the n'th scattering in the same geometry         |
| order_volume_process       | Record only the n'th scattering in same geometry and process |

### Model of the BIFROST collimation + filter using Union_logger_2D_space
![alt](figures/BIFROST_filter.png)

### Adding a few logger components to our example
Back to our example with the diamond sphere within the aluminium shell.

In [12]:
logger_time = instrument.add_component("logger_time_all", "Union_logger_1D", before=master)
logger_time.set_parameters(min_value=0, max_value=0.003, n1=600, filename='"scattering_time.dat"')

logger_time = instrument.add_component("logger_time_shell", "Union_logger_1D", before=master)
logger_time.set_parameters(min_value=0, max_value=0.003, n1=600, filename='"scattering_time_shell.dat"',
                           target_geometry='"shell"')

logger_zx = instrument.add_component("logger_space_zx", "Union_logger_2D_space",
                                     before=master, RELATIVE=sphere)
logger_zx.set_parameters(D_direction_1='"z"', D1_min=-0.07, D1_max=0.07, n1=300,
                         D_direction_2='"x"', D2_min=-0.07, D2_max=0.07, n2=300,
                         filename='"logger_zx.dat"')

logger_zx = instrument.add_component("logger_space_zx_1", "Union_logger_2D_space",
                                     before=master, RELATIVE=sphere)
logger_zx.set_parameters(D_direction_1='"z"', D1_min=-0.07, D1_max=0.07, n1=300,
                         D_direction_2='"x"', D2_min=-0.07, D2_max=0.07, n2=300,
                         filename='"logger_zx_1.dat"', order_total=1)

logger_zx = instrument.add_component("logger_space_zx_2", "Union_logger_2D_space",
                                     before=master, RELATIVE=sphere)
logger_zx.set_parameters(D_direction_1='"z"', D1_min=-0.07, D1_max=0.07, n1=300,
                         D_direction_2='"x"', D2_min=-0.07, D2_max=0.07, n2=300,
                         filename='"logger_zx_2.dat"', order_total=2)

### Adding a few more logger components to our example

In [13]:
logger_zy = instrument.add_component("logger_space_zy", "Union_logger_2D_space",
                                     before=master, RELATIVE=sphere)
logger_zy.set_parameters(D_direction_1='"z"', D1_min=-0.07, D1_max=0.07, n1=300,
                         D_direction_2='"y"', D2_min=-0.07, D2_max=0.07, n2=300,
                         filename='"logger_zy.dat"')

logger_xy = instrument.add_component("logger_space_xy", "Union_logger_2D_space",
                                     before=master, RELATIVE=sphere)
logger_xy.set_parameters(D_direction_1='"x"', D1_min=-0.06, D1_max=0.07, n1=300,
                         D_direction_2='"y"', D2_min=-0.07, D2_max=0.07, n2=300,
                         filename='"logger_xy.dat"') 

logger_2DQ = instrument.add_component("logger_2DQ", "Union_logger_2DQ", before=master)
logger_2DQ.set_parameters(Q_direction_1 = '"z"', Q1_min = -8.0, Q1_max = 8.0, n1 = 500,
                         Q_direction_2 = '"x"', Q2_min = -8.0, Q2_max = 8.0, n2 = 500,
                         filename = '"logger_2DQ.dat"')

L_monitor = instrument.add_component("L_monitor", "L_monitor")
L_monitor.set_AT(0.3, RELATIVE=sphere)
L_monitor.set_parameters(xwidth=0.8, yheight=0.8,
                         Lmin="wavelength - wavelength_half_width",
                         Lmax="wavelength + wavelength_half_width",
                         nL=300, filename='"l_mon.dat"', restore_neutron=1)

banana_detector = instrument.add_component("banana_detector", "Monitor_nD")
banana_detector.set_RELATIVE(sphere)
banana_detector.set_parameters(xwidth = 0.12, yheight = 0.2, filename='"banana_tof.dat"', restore_neutron=1,
                               options='"banana, theta limits=[-180,180] bins=361, t limits=[0.0004 0.00085] bins=500"')

## Running the simulation

In [14]:
import mcstasscript.jb_interface as ms_widget
ms_widget.show(instrument)

### Making an animation from time logger 

In [15]:
logger_zy = instrument.add_component("logger_2D_space_time", "Union_logger_2D_space_time",
                                     before=master, RELATIVE=sphere)
logger_zy.set_parameters(D_direction_1='"z"', D1_min=-0.07, D1_max=0.07, n1=300,
                         D_direction_2='"y"', D2_min=-0.07, D2_max=0.07, n2=300,
                         time_bins=70, time_min=0.00055, time_max=0.00085,
                         filename='"logger_zy_time.dat"')

instrument.set_parameters(wavelength=2.38, wavelength_half_width=0.03)
instrument.settings(ncount=5E7, mpi=6)
data = instrument.backengine()

INFO: Using directory: "/Users/madsbertelsen/McStas/McStas_teaching/Schools/2022/ESS_May_June_2022/Day3_Monday_July_4th/08_Union/Union_demo_data_2"
INFO: Regenerating c-file: Union_demo.c
CFLAGS= -I@MCCODE_LIB@/share/
INFO: Recompiling: ./Union_demo.out
mccode-r.c:1880:1: warning: non-void function does not return a value in all control paths [-Wreturn-type]
} /* mcsiminfo_init */
^
mccode-r.c:2837:3: warning: expression result unused [-Wunused-value]
  *t0;
  ^~~
In file included from /Applications/McStas-2.7.1.app/Contents/Resources/mcstas/2.7.1//contrib/union/Incoherent_process.comp:65:
./Union_functions.c:1604:105: warning: incompatible pointer types passing 'int (const struct saved_history_struct *, const struct saved_history_struct *)' to parameter of type 'int (* _Nonnull)(const void *, const void *)' [-Wincompatible-pointer-types]
  qsort(total_history.saved_histories,total_history.used_elements,sizeof (struct saved_history_struct), Sample_compare_history_intensities);
        

In [16]:
ani_data = ms.name_search("logger_2D_space_time", data)
    
ms.make_animation(ani_data, filename="animation_demo", fps=3,
                  log=True, colormap="hot", orders_of_mag=6, figsize=(8, 5))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

2022-07-03 21:40:28,979 WARNING:MovieWriter imagemagick unavailable; using Pillow instead.


Plotting data with name logger_2D_space_time
Saving animation with filename : "animation_demo.gif"


### Cryostat simulation
More detailed simulation with higher ncount performed on cluster
<table><tr>
<td> <img src="figures/cryostat_geometry.png" alt="Cryostat sketch and data" style="width: 700px;"/> </td>
<td> <img src="figures/output.gif" alt="Cryostat animation" style="width: 1000px;"/> </td>
</tr></table>

### Pressure cell with 2 diamonds and powder sample
Tof pulse around diamond bragg peak
![alt](figures/anvil_bragg.gif)

### Simulated detector signal when rotating one of the diamonds
![alt](figures/full_rotation_movie_2.gif)

## Absorption logger components
Like logger components, but record absorption instead.

| Abs_logger component                    | Dimensions                                |
| :---                                    |    :----                                  |
| Union_abs_logger_1D_space               | 1D space                                  |
| Union_abs_logger_1D_space_event         | 2D space in event format                  |
| Union_abs_logger_1D_space_tof           | 1D space and time                         |
| Union_abs_logger_1D_space_tof_to_lambda | 1D space and time converted to wavelength |
| Union_abs_logger_2D_space               | 2D space                                  |
| Union_abs_logger_event                  | Events  (position, velocity, weight)      |

### Simple model of He3 detector tube
<table><tr>
<td> <img src="figures/power_point_figures/Slide3.png" alt="Detector sketch" style="width: 500px;"/> </td>
<td> <img src="figures/output_absorption_point.gif" alt="Animation" style="width: 900px;"/> </td>
</tr></table>

## Conditional components
Conditional components places a condition on each ray recorded by a logger. The condition applies the the final state of the ray after the Union system, the logger only records the events from that ray if the condition is true.

| Conditional component                   | Condition                                          |
| :---                                    |    :----                                           |
| Union_conditional_standard              | time, energy and scattering order limits           |
| Union_conditional_PSD                   | requires hitting a rectangular area and time limit |

### Adding monitors the conditional component will modify

In [17]:
logger_zx = instrument.add_component("logger_space_zx_con", "Union_logger_2D_space",
                                     before=master, RELATIVE=sphere)
logger_zx.set_parameters(D_direction_1='"z"', D1_min=-0.07, D1_max=0.07, n1=300,
                         D_direction_2='"x"', D2_min=-0.07, D2_max=0.07, n2=300,
                         filename='"logger_zx_con.dat"')

logger_2DQ = instrument.add_component("logger_2DQ_con", "Union_logger_2DQ", before=master)
logger_2DQ.set_parameters(Q_direction_1 = '"z"', Q1_min = -8.0, Q1_max = 8.0, n1 = 500,
                         Q_direction_2 = '"x"', Q2_min = -8.0, Q2_max = 8.0, n2 = 500,
                         filename = '"logger_2DQ_con.dat"')

### Adding conditional component

In [18]:
# Set up instrument parameters describing what spot to investigate
instrument.add_parameter("tag_angle", value=60)
instrument.add_parameter("tag_time", value=0.00072)
instrument.add_parameter("tag_interval", value=0.00005)

# Set up an arm pointing to the relevant spot
spot_dir = instrument.add_component("spot_dir", "Arm", RELATIVE=sphere, before=master)
spot_dir.set_ROTATED([0, "tag_angle", 0], RELATIVE=sphere)

# Set up a conditional component targeting all our loggers
PSD_conditional = instrument.add_component("space_all_PSD_conditional", "Union_conditional_PSD", before=master)
PSD_conditional.set_AT([0, 0, 0.06], RELATIVE=spot_dir) 
PSD_conditional.set_parameters(target_loggers='"logger_space_zx_con,logger_2DQ_con"',
                               xwidth=0.02, yheight=0.2,
                               time_min="tag_time - 0.5*tag_interval",
                               time_max="tag_time + 0.5*tag_interval")

# Extract flag from master
instrument.add_declare_var("int", "flag1")
logger_zx.logger_conditional_extend_index = 1
master.append_EXTEND("flag1 = logger_conditional_extend_array[1];")

# Copy of our banana detector, but with WHEN condition to verify we are investigating the right peak
banana_detector = instrument.add_component("banana_detector_limited", "Monitor_nD")
banana_detector.set_RELATIVE(sphere)
banana_detector.set_parameters(xwidth = 0.12, yheight = 0.2,
                               filename='"banana_tof_limited.dat"', restore_neutron=1,
                               options='"banana, theta limits=[-180,180] bins=361, t limits=[0.0004 0.00085] bins=500"')
banana_detector.set_WHEN("flag1 > 0")

### Running the simulation

In [19]:
logger_zy.time_bins = 5
ms_widget.show(instrument)

# Union overview
In total there are 8 classes of Union components that can be linked together in order to perform the simulation.

| Component class         | Links to                       | Purpose                                    |
| :---                    |    :----                       |    :----                                   |
| Process components      |                                | Describe scattering process                |
| Union_make_material     | List of process components     | Collect process and absorption to material |
| Geometry components     | Material definition            | Place a geometry with selected material    |
| Logger components       | Optionally geometry / process  | Record scattering in Union system          |
| Abs_logger components   | Optionally geometry / process  | Record absorption in Union system          |
| Conditional components  | Logger or Abs_logger           | Place final condition on recording         |
| Union_master            | Geometries (automatic linking) | Simulation engine                          |
| Union_init / Union_stop |                                | Needed in McStas 3.X at start / end        |



## Conclusion
The Union components is a powerful addition to McStas that allow complex simulations with simple tools. The advantages are:
- Splits responsibility into geometry, physics and simulation engine
- Modular physics description (expandable)
- Allows overlapping geometry and hollow systems like cryostats
- Full multiple scattering
- Comprehensive logging tools
- System for tracking interesting features (Conditional components)
- Can be used to simulate detector systems

Still to do:
- Polarization not supported
- Gravity not supported
- Surface effects

## Quiz and exercise
Now its time for you to do a quiz about the Union components and use them to build an instrument. The quiz is called Union_quiz and is available in the McStasScript-notebooks. If only the solution is available, a git pull will be needed from the McStasScript-notebooks folder.